In [15]:
import os
import pandas as pd
import numpy as np
import librosa
import scipy.signal as signal
import matplotlib.pyplot as plt
from datetime import datetime

ModuleNotFoundError: No module named 'librosa'

In [16]:
!pip install librosa

  Using cached librosa-0.10.2.post1-py3-none-any.whl.metadata (8.6 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached soundfile-0.12.1-py2.py3-none-macosx_11_0_arm64.whl.metadata (14 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
Using cached librosa-0.10.2.post1-py3-none-any.whl (260 kB)
Using cached audioread-3.0.1-py3-none-any.whl (23 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached lazy_loader-0.4-

In [6]:
input_csv_path = '/Users/hashimmuhammadnadeem/learning-ai-with-baseer/internship/data/output-average-loudness_20241126_160914.csv'

if not os.path.exists(input_csv_path):
    raise FileNotFoundError(f"The file '{input_csv_path}' does not exist in the current directory.")

df = pd.read_csv(input_csv_path)

df.head()


,Filename,Turbine ID,Datetime,Bin1 Power,Bin2 Power,Bin3 Power
0,2024-08-06 15_30_10.228358Windrover.wav,soundfiles,2024-08-06 15:30:10.228358,-74.998492,-82.934939,-89.882636
1,2024-08-06 17_30_14.844987Windrover.wav,soundfiles,2024-08-06 17:30:14.844987,-78.462828,-82.719445,-99.106127
2,2024-08-06 19_30_09.896151Windrover.wav,soundfiles,2024-08-06 19:30:09.896151,-75.707854,-93.939137,-100.847255
3,2024-08-06 21_30_14.248628Windrover.wav,soundfiles,2024-08-06 21:30:14.248628,-73.483013,-93.815900,-100.868039
4,2024-08-06 23_30_08.454071Windrover.wav,soundfiles,2024-08-06 23:30:08.454071,-68.382399,-90.957999,-100.812166


In [8]:
print("\nMissing values in each column:")
print(df.isnull().sum())


Missing values in each column:
Filename      0
Turbine ID    0
Datetime      0
Bin1 Power    0
Bin2 Power    0
Bin3 Power    0
dtype: int64


In [9]:
def extract_turbine_id(filename):
    """
    Extracts the Turbine ID from the given filename.

    Parameters:
    filename (str): The filename string, e.g., "2024-08-06 15_30_10.228358Windrover.wav"

    Returns:
    str: Extracted Turbine ID, e.g., "228358Windrover.wav"
    """
    try:
        # Split the filename at the first '.' character
        parts = filename.split('.', 1)
        if len(parts) == 2:
            turbine_id_with_ext = parts[1]
            return turbine_id_with_ext
        else:
            # If there's no '.', return 'Unknown' or handle as needed
            return 'Unknown'
    except Exception as e:
        print(f"Error extracting Turbine ID from filename '{filename}': {e}")
        return 'Error'

In [12]:
df['Turbine ID'] = df['Filename'].apply(extract_turbine_id)

df

,Filename,Turbine ID,Datetime,Bin1 Power,Bin2 Power,Bin3 Power
0,2024-08-06 15_30_10.228358Windrover.wav,228358Windrover.wav,2024-08-06 15:30:10.228358,-74.998492,-82.934939,-89.882636
1,2024-08-06 17_30_14.844987Windrover.wav,844987Windrover.wav,2024-08-06 17:30:14.844987,-78.462828,-82.719445,-99.106127
2,2024-08-06 19_30_09.896151Windrover.wav,896151Windrover.wav,2024-08-06 19:30:09.896151,-75.707854,-93.939137,-100.847255
3,2024-08-06 21_30_14.248628Windrover.wav,248628Windrover.wav,2024-08-06 21:30:14.248628,-73.483013,-93.815900,-100.868039
4,2024-08-06 23_30_08.454071Windrover.wav,454071Windrover.wav,2024-08-06 23:30:08.454071,-68.382399,-90.957999,-100.812166
...,...,...,...,...,...,...
1743,2024-10-21 10_40_43.423409Windrover.wav,423409Windrover.wav,2024-10-21 10:40:43.423409,-58.801222,-83.550773,-98.677030
1744,2024-10-21 11_40_15.968269Windrover.wav,968269Windrover.wav,2024-10-21 11:40:15.968269,-60.719371,-83.703258,-89.892303
1745,2024-10-21 12_40_11.179473Windrover.wav,179473Windrover.wav,2024-10-21 12:40:11.179473,-65.843213,-89.885570,-99.580519
1746,2024-10-21 13_40_10.747145Windrover.wav,747145Windrover.wav,2024-10-21 13:40:10.747145,-69.622298,-91.971991,-99.678832


In [11]:
unique_turbine_ids = df['Turbine ID'].unique()
print(f"Number of unique Turbine IDs: {len(unique_turbine_ids)}")
print("Unique Turbine IDs:")
print(unique_turbine_ids)

Number of unique Turbine IDs: 1747
Unique Turbine IDs:
['228358Windrover.wav' '844987Windrover.wav' '896151Windrover.wav' ...
 '179473Windrover.wav' '747145Windrover.wav' '674170Windrover.wav']


In [13]:
problematic_rows = df[df['Turbine ID'].isin(['Unknown', 'Error'])]

if not problematic_rows.empty:
    print("\nRows with problematic Turbine IDs:")
    display(problematic_rows)
else:
    print("\nAll Turbine IDs have been correctly extracted.")


All Turbine IDs have been correctly extracted.


As we saw earlier, there was not any missing row, hence no problamatic turbine ids.

In [14]:
df

,Filename,Turbine ID,Datetime,Bin1 Power,Bin2 Power,Bin3 Power
0,2024-08-06 15_30_10.228358Windrover.wav,228358Windrover.wav,2024-08-06 15:30:10.228358,-74.998492,-82.934939,-89.882636
1,2024-08-06 17_30_14.844987Windrover.wav,844987Windrover.wav,2024-08-06 17:30:14.844987,-78.462828,-82.719445,-99.106127
2,2024-08-06 19_30_09.896151Windrover.wav,896151Windrover.wav,2024-08-06 19:30:09.896151,-75.707854,-93.939137,-100.847255
3,2024-08-06 21_30_14.248628Windrover.wav,248628Windrover.wav,2024-08-06 21:30:14.248628,-73.483013,-93.815900,-100.868039
4,2024-08-06 23_30_08.454071Windrover.wav,454071Windrover.wav,2024-08-06 23:30:08.454071,-68.382399,-90.957999,-100.812166
...,...,...,...,...,...,...
1743,2024-10-21 10_40_43.423409Windrover.wav,423409Windrover.wav,2024-10-21 10:40:43.423409,-58.801222,-83.550773,-98.677030
1744,2024-10-21 11_40_15.968269Windrover.wav,968269Windrover.wav,2024-10-21 11:40:15.968269,-60.719371,-83.703258,-89.892303
1745,2024-10-21 12_40_11.179473Windrover.wav,179473Windrover.wav,2024-10-21 12:40:11.179473,-65.843213,-89.885570,-99.580519
1746,2024-10-21 13_40_10.747145Windrover.wav,747145Windrover.wav,2024-10-21 13:40:10.747145,-69.622298,-91.971991,-99.678832
